```
p_паттерн парсит текст (str, pos), 
    вызывая другие паттерны, возвращающие древовидные структуры
    эти древовидные структуры передает одному из правил
        (остальные вызовы правил закомментированы 
            - в рамках одного текста каждый паттерн имеет ровно 1 смысл)
    и возвращает (pos, результат этого правила), помещенный в массив
    
    /*пока на ошибках парсинга концентрировать не будем*/
    если ничего не удалось распарсить, ???возвращаемый массив будет пустым???
    если удалось распарсить несколько вариантов - в массиве будет насколько вариантов
        сначала парсятся все исключения
        потом парсятся все обычные варианты (если нет исключений)
```

```
r_правило получает список древовидных структур
    обрабатывает их по определенному правилу
    возвращает древовидную структуру
```

```
древовидная структура - map, со следующими ключами
    type: 'noun'/'adj'/'verb'/'num'/... - определяем через isinstance()
    постоянные параметры (для сущ.: род, число)
    переменные параметры (для сущ.: падеж)
    talk: массив древовидных структур
        или пар (тип, слово), где тип - main/punct/other
```

```
str(древовидная структура)
    превращает древовидную структуру в строку
```

```
для каждого типа элемента древовидной структуры будет map
    где каждой строке (для сущ. - слово в и.п.) будет соответствовать
        структура данных, которая вместе в переменными параметрами передается в 
        sh_функция для этого типа элемента древовидной структуры
            которая будет возвращать слово в соотв. форме (для сущ. в соотв. падеже)
```

In [1]:
from classes import *

# Правила: Словарь, +отображение

In [2]:
ruwords={} #какие-то однословные правила складываются сюда, а какие-то остаются в функциях

## Noun

##### Функции

In [3]:
def show_noun1(st,word,ip,rp,dp,vp,tp,pp):
    if st.pad=='ip' : return word+ip
    if st.pad=='rp' : return word+rp
    if st.pad=='dp' : return word+dp
    if st.pad=='vp' : return word+vp
    if st.pad=='tp' : return word+tp
    if st.pad=='pp' : return word+pp
    raise RuntimeError()


In [4]:
#show_noun_map['я']=lambda st: show_noun1(st,"","я","меня","мне","меня","мной","мне")


In [5]:
def add_runoun(name,och,odush,rod,chis,word,ip,rp,dp,vp,tp,pp):
    ruwords[name]=StNoun(name,och,odush,rod,chis,'ip')
    show_noun_map[name]=lambda st: show_noun1(st,word,ip,rp,dp,vp,tp,pp)

def add_runoun2(name,rod,word,ip,rp,dp,vp,tp,pp,\
               mname,odush,mword,mip,mrp,mdp,mvp,mtp,mpp):
    add_runoun(name,mname,odush,rod,'ed',word,ip,rp,dp,vp,tp,pp)
    add_runoun(mname,name,odush,rod,'mn',mword,mip,mrp,mdp,mvp,mtp,mpp)


##### М.р.

In [6]:
#                                     ип ,нет ,дать,вижу,творю,думаю
add_runoun2("кот"    ,'m'  ,'кот'    ,'' ,'а' ,'у' ,'а' ,'ом' ,'е' ,
            "коты"   ,True ,'кот'    ,'ы','ов','ам','ов','ами','ах')

add_runoun2("джем"   ,'m'  ,'джем'   ,'' ,'а' ,'у' ,''  ,'ом' ,'е' ,
            "джемы"  ,False,'джем'   ,'ы','ов','ам','ы' ,'ами','ах')

add_runoun2("волк"   ,'m'  ,'волк'   ,'' ,'а' ,'у' ,'а' ,'ом' ,'е' ,
            "волки"  ,True ,'волк'   ,'и','ов','ам','ов','ами','ах')
add_runoun2("мальчик",'m'  ,'мальчик','' ,'а' ,'у' ,'а' ,'ом' ,'е' ,
            "мальчики",True,'мальчик','и','ов','ам','ов','ами','ах')

add_runoun2("ящик"   ,'m'  ,'ящик'   ,'' ,'а' ,'у' ,''  ,'ом' ,'е' ,
            "ящики"  ,False,'ящик'   ,'и','ов','ам','и' ,'ами','ах')
add_runoun2("урок"   ,'m'  ,'урок'   ,'' ,'а' ,'у' ,''  ,'ом' ,'е' ,
            "уроки"  ,False,'урок'   ,'и','ов','ам','и' ,'ами','ах')

In [7]:
add_runoun2("ребёнок",'m'  ,'ребён'  ,'ок','ка','ку','ка','ком','ке',
            "дети"   ,True ,'дет'    ,'и' ,'ей','ям','ей','ьми','ях')

add_runoun2("котёнок",'m'  ,'котён'  ,'ок','ка','ку','ка','ком','ке',
            "котята" ,True ,'котят'  ,'а' ,''  ,'ам',''  ,'ами','ах')

##### Ж.р.

In [8]:
#                                   ип ,нет ,дать ,вижу,творю ,думаю
add_runoun2("белка" ,'g'  ,'белк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "белки" ,True ,'бел'  ,'ки','ок','кам','ок','ками','ках')

add_runoun2("кепка"  ,'g'  ,'кепк'  ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "кепки"  ,False,'кеп'   ,'ки','ок','кам','ки','ками','ках')
add_runoun2("шапка"  ,'g'  ,'шапк'  ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "шапки"  ,False,'шап'   ,'ки','ок','кам','ки','ками','ках')
add_runoun2("коробка",'g'  ,'коробк','а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "коробки",False,'короб' ,'ки','ок','кам','ки','ками','ках')

add_runoun2("кошка" ,'g'  ,'кошк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "кошки" ,True ,'кош'  ,'ки','ек','кам','ек','ками','ках')

add_runoun2("ручка" ,'g'  ,'ручк' ,'а' ,'и' ,'е'  ,'у'  ,'ой' ,'е'  ,
            "ручки" ,False,'руч'  ,'ки','ек','кам','ки','ками','ках')
add_runoun2("чашка" ,'g'  ,'чашк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "чашки" ,False,'чаш'  ,'ки','ек','кам','ки','ками','ках')
add_runoun2("ложка" ,'g'  ,'ложк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "ложки" ,False,'лож'  ,'ки','ек','кам','ки','ками','ках')

In [9]:
add_runoun2("собака",'g'  ,'собак','а' ,'и' ,'е'  ,'у'  ,'ой' ,'е' ,
            "собаки",True ,'собак','и' ,''  ,'ам' ,''   ,'ами','ах')

add_runoun2("книга" ,'g'  ,'книг' ,'а' ,'и' ,'е'  ,'у'  ,'ой' ,'е' ,
            "книги" ,False,'книг' ,'и' ,''  ,'ам' ,'и'  ,'ами','ах')

add_runoun2("зебра" ,'g'  ,'зебр' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "зебры" ,True ,'зебр' ,'ы','' ,'ам',''  ,'ами','ах')
add_runoun2("крыса" ,'g'  ,'крыс' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "крысы" ,True ,'крыс' ,'ы','' ,'ам',''  ,'ами','ах')
add_runoun2("курица",'g'  ,'куриц','а','ы','е' ,'у' ,'ой' ,'е' ,
            "курицы",True ,'куриц','ы','' ,'ам',''  ,'ами','ах')
add_runoun2("лиса"  ,'g'  ,'лис'  ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "лисы"  ,True ,'лис'  ,'ы','' ,'ам',''  ,'ами','ах')
    
add_runoun2("шляпа" ,'g'  ,'шляп' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "шляпы" ,False,'шляп' ,'ы','' ,'ам','ы' ,'ами','ах')
add_runoun2("ваза"  ,'g'  ,'ваз'  ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "вазы"  ,False,'ваз'  ,'ы','' ,'ам','ы' ,'ами','ах')
add_runoun2("лампа" ,'g'  ,'ламп' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "лампы" ,False,'ламп' ,'ы','' ,'ам','ы' ,'ами','ах')
add_runoun2("звезда",'g'  ,'звезд','а','ы','е' ,'у' ,'ой' ,'е' ,
            "звёзды",False,'звёзд','ы','' ,'ам','ы' ,'ами','ах')

In [10]:
add_runoun2("свинья",'g'  ,'свинь','я' ,'и' ,'е'  ,'ю' ,'ёй'  ,'е'  ,
            "свиньи",True ,'свин' ,'ьи','ей','ьям','ей','ьями','ьях')

In [11]:
add_runoun2("мышь"   ,'g'  ,'мыш'   ,'ь','и' ,'и' ,'ь' ,'ью' ,'и' ,
            "мыши"   ,True ,'мыш'   ,'и','ей','ам','ей','ами','ах')

add_runoun2("тетрадь",'g'  ,'тетрад','ь','и' ,'и' ,'ь' ,'ью' ,'и' ,
            "тетради",False,'тетрад','и','ей','ям','и' ,'ями','ях')
add_runoun2("кровать",'g'  ,'кроват','ь','и' ,'и' ,'ь' ,'ью' ,'и' ,
            "кровати",False,'кроват','и','ей','ям','и' ,'ями','ях')

##### С.р.

In [12]:
#                                    ип  ,нет ,дать ,вижу,творю ,думаю о
add_runoun2("утро"  ,'s'  ,'утр'    ,'о' ,'а' ,'у'  ,'о' ,'ом'  ,'е'  ,
            "утра"  ,False,'утр'    ,'а' ,''  ,'ам' ,'а' ,'ами' ,'ах' )

add_runoun2("ружьё"  ,'s'  ,'ружь'  ,'ё' ,'я' ,'ю'  ,'ё' ,'ём'  ,'е'  ,
            "ружья"  ,False,'руж'   ,'ья','ей','ьям','ьи','ьями','ьях')

add_runoun2("вареньё",'s'  ,'варень','е' ,'я' ,'ю'  ,'е' ,'ем' ,'е' ,
            "варенья",False,'варен' ,'ья','ий','ьям','ья','ьями','ьях')

##### Мн.ч.

In [13]:
#                                  ип ,нет ,дать,вижу,творю,думаю
add_runoun('часы (предмет)',None,False,'m','mn','час','ы','ов','ам','ы','ами','ах')

## Pronoun

In [14]:
ruwords["я (муж)"]=StProNoun("я","мы",1,True,'m','ed','ip')
ruwords["я (жен)"]=StProNoun("я","мы",1,True,'g','ed','ip')
show_noun_map["я"]=lambda st: show_noun1(st,'',"я","меня","мне","меня","мной","мне")

## Adj

In [15]:
def show_adj1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp,sh)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp,sh)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    elif st.pad=='sh' : rez=sh
    else: raise RuntimeError()
        
    return word+rez


In [16]:
def add_ruadj(name,sh_word,ends):
    ruwords[name]=StAdj(name,True,'m','ed','ip')
    show_adj_map[name]=lambda st: show_adj1(st,sh_word,ends)


In [17]:
adj_std_ends_i={# и
    'm' :('ий','его','ему',('его','ий'),'им' ,'ем','' ),
    's' :('ее','его','ему',('ее', 'ее'),'им' ,'ем','е'),
    'g' :('ая','ей' ,'ей' ,('ую', 'ую'),'ей' ,'ей','а'),
    'mn':('ие','их' ,'им' ,('их', 'ие'),'ими','их','и'),
}
adj_std_ends_y={# ы
    'm' :('ый','ого','ому',('ого','ый'),'ым' ,'ом','' ),
    's' :('ое','ого','ому',('ое', 'ое'),'ым' ,'ом','о'),
    'g' :('ая','ой' ,'ой' ,('ую', 'ую'),'ой' ,'ой','а'),
    'mn':('ые','ых' ,'ым' ,('ых', 'ые'),'ыми','ых','ы'),
}

add_ruadj('летучий','летуч' ,adj_std_ends_i)
add_ruadj('хороший','хорош' ,adj_std_ends_i)

add_ruadj('добрый' ,'добр' ,adj_std_ends_y)


## Num

In [18]:
def show_num1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    else: raise RuntimeError()
        
    return word+rez


In [19]:
show_num_map['один']=\
    lambda st: show_num1(st,'од' ,
        {
            'm' :('ин','ного','ному',('ного','ин'),'ним' ,'ном'),
            's' :('но','ного','ному',('но'  ,'но'),'ним' ,'ном'),
            'g' :('на','ной' ,'ной' ,('ну'  ,'ну'),'ной' ,'ной'),
            'mn':('ни','них' ,'ним' ,('них', 'ни'),'ними','них'),
        }
    )
show_num_map['два']=\
    (lambda st: \
        ("две" if st.rod=='g' else "два")                         if st.pad=='ip' else \
        "двух"                                                    if st.pad=='rp' else \
        "двум"                                                    if st.pad=='dp' else \
        ("двух" if st.odush else "две" if st.rod=='g' else "два") if st.pad=='vp' else \
        "двумя"                                                   if st.pad=='tp' else \
        "двух"                                                    if st.pad=='pp' else \
        throw(RuntimeError('unknown pad: '+st.pad))
    )
show_num_map['три']=\
    lambda st: show_num1(st,'тр' ,
        {   'mn':('и','ёх' ,'ём' ,('ёх', 'и'),'емя','ёх'), }    )
show_num_map['четыре']=\
    lambda st: show_num1(st,'четыр' ,
        {   'mn':('е','ёх' ,'ём' ,('ёх', 'е'),'ьмя','ёх'), }    )
show_num_map['пять']=\
    lambda st: show_num1(st,'пят' ,
        {   'mn':('ь','и' ,'и' ,('ь', 'ь'),'ью','и'), }    )
show_num_map['шесть']=\
    lambda st: show_num1(st,'шест' ,
        {   'mn':('ь','и' ,'и' ,('ь', 'ь'),'ью','и'), }    )
show_num_map['семь']=\
    lambda st: show_num1(st,'сем' ,
        {   'mn':('ь','и' ,'и' ,('ь', 'ь'),'ью','и'), }    )


In [20]:
def r_odin():    return StNum('один'  ,'1'  ,False,'m','ed','ip')
def r_dva():     return StNum('два'   ,'2-4',False,'m','mn','ip')
def r_tri():     return StNum('три'   ,'2-4',False,'m','mn','ip')
def r_chetyre(): return StNum('четыре','2-4',False,'m','mn','ip')
def r_pyat():    return StNum('пять'  ,'>=5',False,'m','mn','ip')
def r_shest():   return StNum('шесть' ,'>=5',False,'m','mn','ip')
def r_sem():     return StNum('семь'  ,'>=5',False,'m','mn','ip')


## Verb

In [21]:
def show_verb1(st,word,end_map):
    if st.form=='neopr':
        end = end_map[st.form]
    elif  st.form=='povel':
        end = end_map[st.form][0 if st.chis=='ed' else 1]
    else:
        (i,you,he,shi,it,we,yous,they)=end_map[st.form]
        if   (st.pers,st.chis)==(1,'ed'):
            end = i
        elif (st.pers,st.chis)==(2,'ed'):
            end = you
        elif (st.pers,st.chis)==(3,'ed'):
            if   st.rod=='m' : end=he
            elif st.rod=='g' : end=she
            elif st.rod=='s' : end=it
        elif (st.pers,st.chis)==(1,'mn'):
            end = we
        elif (st.pers,st.chis)==(2,'mn'):
            end = yous
        elif (st.pers,st.chis)==(3,'mn'):
            end = they
    return word+end

In [22]:
show_verb_map['видеть']=\
    lambda st: show_verb1(st,'ви',
        {    'neopr':"деть",
             'povel':("дь","дьте"),
             'nast': ("жу","дишь","дит","дит","дит","дим","дите","дят")
        }   )
show_verb_map['иметь']=\
    lambda st: show_verb1(st,'име',
        {    'neopr':"ть",
             'povel':("й","йте"),
             'nast': ("ю","ешь","ет","ет","ет","ем","ете","ют")
        }   )
show_verb_map['показать']=\
    lambda st: show_verb1(st,'пока',
        {    'neopr':"зать",
             'povel':("жи","жите"),
             'nast': ("жу","жешь","жет","жет","жет","жем","жете","жут")
        }   )
show_verb_map['дать']=\
    lambda st: show_verb1(st,'да',
        {    'neopr':"ть",
             'povel':("й","йте"),
             'nast': ("м","шь","ст","ст","ст","дим","дите","дут")
        }   )

In [23]:
#(self,word,oasp=0,asp=None,form=None,chis=0,rod=0,pers=0)
def r_videt():   return StVerb('видеть'  ,"увидеть"   ,'nesov','povel','mn',None,None)
def r_imet():    return StVerb('иметь'   ,"заиметь"   ,'nesov','povel','mn',None,None)
def r_dat():     return StVerb('дать'    ,'давать'    ,'sov'  ,'povel','mn',None,None)
def r_pokazat(): return StVerb('показать','показывать','sov'  ,'povel','mn',None,None)

# Правила: Составные - общие

In [24]:
def r_adj_noun(_a_,_n_): 
    return StNoun([
        I(dep=_a_,
            rod=_n_.rod,
            chis=_n_.chis,
            pad=_n_.pad),
        I(maindep=_n_)
    ])

# Паттерны: Словарь

## Adj

In [25]:
dict_adj={}

In [26]:
dict_adj['a'] = S('')
#dict_adj['a'] : r_nekotoryj(),
dict_adj['an']= S('')
#dict_adj['an'] : r_nekotoryj(),

dict_adj['good']=ruwords["хороший"]


## Noun

In [27]:
dict_noun={}

In [28]:
#dict_noun['cat']=   ruwords["кошка"]
dict_noun['cat']=   ruwords["кот"]
#dict_noun['cats']=   ruwords["кошки"]
dict_noun['cats']=   ruwords["коты"]
    
#dict_noun['rat']=   ruwords["мышь"]
dict_noun['rat']=   ruwords['крыса']
#dict_noun['rats']=   ruwords["мыши"]
dict_noun['rats']=   ruwords['крысы']
    
dict_noun['bat']=   r_adj_noun(deepcopy(ruwords['летучий']),deepcopy(ruwords["мышь"]))
dict_noun['bats']=   r_adj_noun(deepcopy(ruwords['летучий']),deepcopy(ruwords["мыши"]))
    
dict_noun['lesson']=ruwords["урок"]
dict_noun['lessons']=ruwords["уроки"]
    

In [29]:
dict_noun['cap']=   ruwords["кепка"]
#dict_noun['cap']=  ruwords["шапка"]
dict_noun['caps']=   ruwords["кепки"]
#dict_noun['caps']=  ruwords["шапки"]
    
dict_noun['pen']=   ruwords["ручка"]
dict_noun['pens']=   ruwords["ручки"]
    
dict_noun['hat']=   ruwords["шляпа"]
dict_noun['hats']=   ruwords["шляпы"]
    
dict_noun['hen']=   ruwords["курица"]
dict_noun['hens']=   ruwords["курицы"]
    

In [30]:
dict_noun['dog']=   ruwords['собака']
dict_noun['dogs']=   ruwords['собаки']
dict_noun['pig']=   ruwords['свинья']
dict_noun['pigs']=   ruwords['свиньи']
dict_noun['gun']=   ruwords['ружьё']
dict_noun['guns']=   ruwords['ружья']
dict_noun['cup']=   ruwords['чашка']
dict_noun['cups']=   ruwords['чашки']
    

In [31]:
#dict_noun['box']=   ruwords['коробка']
#dict_noun['boxes']=   ruwords['коробки']
dict_noun['box']=   ruwords['ящик']
dict_noun['boxes']=   ruwords['ящики']

dict_noun['jam']=   ruwords['джем']
#dict_noun['jam']=   ruwords['варенье']

dict_noun['bed']=   ruwords['кровать']
dict_noun['beds']=   ruwords['кровати']

dict_noun['fox']=   ruwords['лиса']
dict_noun['foxes']=   ruwords['лисы']
    

In [32]:
dict_noun['kitten']= ruwords['котёнок']
dict_noun['kittens']= ruwords['котята']
dict_noun['vase']= ruwords['ваза']
dict_noun['vases']= ruwords['вазы']
dict_noun['star']= ruwords['звезда']
dict_noun['stars']= ruwords['звёзды']
dict_noun['lamp']= ruwords['лампа']
dict_noun['lamps']= ruwords['лампы']
    

In [33]:
dict_noun['squirrel']= ruwords['белка']
dict_noun['squirrels']= ruwords['белки']
dict_noun['wolf']= ruwords['волк']
dict_noun['wolfs']= ruwords['волки']
dict_noun['zebra']= ruwords['зебра']
dict_noun['zebras']= ruwords['зебры']
dict_noun['boy']= ruwords['мальчик']
dict_noun['boys']= ruwords['мальчики']
#dict_noun['boy']= ruwords['парень']
#dict_noun['boy']= ruwords['парни']

In [34]:
dict_noun['copy-book']= ruwords['тетрадь']
dict_noun['copy-books']= ruwords['тетради']
dict_noun['book']= ruwords['книга']
dict_noun['books']= ruwords['книги']
dict_noun['spoon']= ruwords['ложка']
dict_noun['spoons']= ruwords['ложки']
dict_noun['morning']= ruwords['утро']
dict_noun['mornings']= ruwords['утра']


In [35]:
dict_noun['child']= ruwords['ребёнок']
dict_noun['children']= ruwords['дети']

dict_noun['watch']= ruwords['часы (предмет)']

## Pronoun

In [36]:
dict_pronoun_ip={}
dict_pronoun_dp={}

In [37]:
dict_pronoun_ip['I']= ruwords["я (муж)"]
#dict_pronoun_ip['I']= ruwords["я (жен)"]

dict_pronoun_dp['me']= ruwords["я (муж)"]
#dict_pronoun_dp['I']= ruwords["я (жен)"]

## Numeral

In [38]:
dict_numeral={}

In [39]:
dict_numeral['one']=   r_odin()
dict_numeral['two']=   r_dva()
dict_numeral['three']= r_tri()
dict_numeral['four']=  r_chetyre()
dict_numeral['five']=  r_pyat()
dict_numeral['six']=   r_shest()
dict_numeral['seven']= r_sem()


## Verb

In [40]:
dict_verb={}
dict_verb_s={}

In [41]:
dict_verb  ['see']= r_videt()
dict_verb_s['sees']= r_videt()
dict_verb  ['have']= r_imet()
dict_verb_s['has']= r_imet()

dict_verb  ['give']= r_dat()
dict_verb_s['gives']= r_dat()
dict_verb  ['show']= r_pokazat()
dict_verb_s['shows']= r_pokazat()


# Паттерны и правила: Составные

In [42]:
DEBUG_PP_PRINTS=False

In [43]:
def debug_pp(fun):
    def wrapper(s,p):
        if DEBUG_PP_PRINTS:
            print('{'+'.'*p+fun.__name__)
            
        rezs=fun(s,p)
        
        if DEBUG_PP_PRINTS:
            print('_'+'.'*p+str(len(rezs)),'in ',fun.__name__,'}',
                  [(p,str(r)) for (p,r) in rezs],'\n')
            for i in rezs:
                if isinstance(i[1],StDeclinable):
                    i[1].check_attrs('wrapper:'+fun.__name__)
        return rezs
    return wrapper
    return fun


## Other

In [44]:
@debug_pp
def p_numeral(s,p):
    return D(dict_numeral)(s,p)

In [45]:
#2->
@debug_pp
def p_adj(s,p):
    return D(dict_adj)(s,p)

## Noun-like

In [46]:
def r_A_noun(a,n): 
    return SAttrs.to_right(a,n)

In [47]:
def r_GOOD_MORNING(g,m):
    _g_=deepcopy(ruwords['добрый'])
    _g_.attrs=g.attrs
    _m_=deepcopy(ruwords['утро'])
    _m_.attrs=m.attrs
    return r_adj_noun(_g_,_m_)

In [48]:
@debug_pp
def p_adj_noun3(s,p): return alt(
    seq([ alt(W('an'),ELSE,W('a')), p_noun3 ],r_A_noun),
    #seq([ alt(W('an'),ELSE,W('a')), p_noun3 ],r_NEKOTORYJ_noun),
    seq([ W('good'), W('morning') ],r_GOOD_MORNING),             ELSE,
    seq([ p_adj, p_noun3 ],r_adj_noun)
)(s,p)

In [49]:
@debug_pp
def p_noun3(s,p): return alt(
    p_adj_noun3,
    p_numeral,
    D(dict_noun),
    D(dict_pronoun_ip)
)(s,p)

In [50]:
def r_noun_numeral(n,num):
    return StNoun([
        I(maindep=n),
        I(nomer=num)
    ])

In [51]:
@debug_pp
def p_noun2(s,p):
    rezs=[]
    rezs+= seq([ p_noun3, p_numeral ], r_noun_numeral)(s,p)
    rezs+= p_noun3(s,p)
    return rezs

In [52]:
def r_numeral_noun(num,n):
    if num.chis!=n.chis :
        print('не совпадают числа числ. и сущ.:',str(num),str(n))
    return StNum([
        I(quantity=num,
            chis=n.chis,
            rod=n.rod,
            odush=n.odush),
        I(maindep=n)
    ],quantity=num.quantity)

In [53]:
@debug_pp
def p_noun1(s,p):
    rezs=[]
    rezs+= seq([ p_numeral, p_noun2 ], r_numeral_noun)(s,p)
    rezs+= p_noun2(s,p)
    return rezs

In [54]:
def r_noun_and_noun(sn,a,n):
    return StNoun([
        I(dep=sn),
        I(nodep=S('и',a.attrs)),
        I(dep=n)
    ],c='mn', p='ip',o=False,r='m')
def r_noun_comma_noun(sn,c,n):
    return StNoun([
        I(dep=sn),
        I(punct=S(',',c.attrs)),
        I(dep=n)
    ],c='mn', p='ip',o=False,r='m')


In [55]:
@debug_pp
def p_noun(s,p):
    @debug_pp
    def p_noun1_and_noun(s,p):
        rezs=  seq([ p_noun1, W('and'), p_noun ],r_noun_and_noun  )(s,p)
        rezs+= seq([ p_noun1, W(',')  , p_noun ],r_noun_comma_noun)(s,p)
        return rezs
    rezs=[]
    rezs+=  p_noun1_and_noun(s,p)
    rezs+= p_noun1(s,p)
    return rezs

In [56]:
def r_noun_dp(_n_):
    _n_.pad='dp'
    return _n_

In [57]:
@debug_pp
def p_noun_dp(s,p): return alt(
    rule1( D(dict_pronoun_dp) ,r_noun_dp), ELSE,
    seq([ W('to'), p_noun ],r_noun_dp,[1])
)(s,p)

## Verb-like

In [58]:
def r_verb_noun_dp_mn(_v_,_n_): 
    return StVerb([
        I(maindep=_v_,
            chis='mn'),
        I(dp=_n_,
            pad='dp')
    ])

In [59]:
def r_verb_noun_dp_ed(_v_,_n_): 
    return StVerb([
        I(maindep=_v_,
            chis='ed'),
        I(dp=_n_,
            pad='dp')
    ])

In [60]:
@debug_pp
def p_verb3(s,p): return alt(
    seq([ D(dict_verb),   p_noun_dp ],r_verb_noun_dp_ed),
    seq([ D(dict_verb_s), p_noun_dp ],r_verb_noun_dp_ed),  ELSE,
#    seq([ D(dict_verb),   p_noun_dp ],r_verb_noun_dp_mn),
#    seq([ D(dict_verb_s), p_noun_dp ],r_verb_noun_dp_mn),  ELSE,
    D(dict_verb),                                       
    D(dict_verb_s)
)(s,p)

In [61]:
def r_verb_noun(v,n): 
    return StVerb([
        I(maindep=v),
        I(vp=n,
            pad='vp')
    ])

In [62]:
@debug_pp
def p_verb2(s,p): return alt(
    seq([ p_verb3, p_noun ],r_verb_noun),    ELSE,
    p_verb3
)(s,p)    

In [63]:
def r_U_noun_EST_noun(_n1_,_h_,_n2_):
    return StC([
        I(nodep=StC([
            I(nodep=S('у')),
            I(nodep=_n1_,
                pad='rp')
        ])),
        I(nodep=S('есть',_h_.attrs)),
        I(nodep=_n2_)
    ])

def r_U_noun_NET_noun(_n1_,_h_,_no_,_n2_):
    return StC([
        I(nodep=StC([
            I(nodep=S('у')),
            I(nodep=_n1_,
                pad='rp')
        ])),
        I(nodep=S('нет',_h_.attrs)),
        I(nodep=_n2_,
            pad='rp')
    ])

In [64]:
@debug_pp
def p_HAVE_noun(s,p):
    p_HAVE_HAS = alt( W('have'), W('has') )
    p_pronoun_dp = alt( D(dict_pronoun_dp), ELSE, p_noun )
    return alt(
        seq([ p_noun, p_HAVE_HAS,          p_pronoun_dp ],r_U_noun_EST_noun),
        seq([ p_noun, p_HAVE_HAS, W('no'), p_pronoun_dp ],r_U_noun_NET_noun)
    )(s,p)

In [65]:
def r_to_verb(t,v): 
    v.form='neopr'
    return SAttrs.to_right(t,v)

In [66]:
def r_noun_verb(n,v): 
    return StVerb([
        I(ip=n),
        I(main=v,
            form='nast',
            pers=n.pers,
            chis=n.chis,
            rod=n.rod)
    ])

In [67]:
@debug_pp
def p_verb1(s,p): return alt(
    p_HAVE_noun,                           ELSE,
    seq([ p_noun, p_verb2 ],r_noun_verb),
    seq([ W('to'), p_verb2 ],r_to_verb),   ELSE,
    p_verb2
)(s,p)

In [68]:
def r_verb_BUT_verb(_v1_,_c_,_but_,_v2_):
    return StC([
        I(nodep=_v1_),
        I(nodep=_c_),
        I(nodep=S('но',_c_.attrs)),
        I(nodep=_v2_)
    ])

In [69]:
@debug_pp
def p_verb(s,p): return alt(
    seq([ p_verb1, W(','), W('but'), p_verb1 ],r_verb_BUT_verb),    ELSE,
    p_verb1
)(s,p)

## Фразы, предложения, текст

In [70]:
@debug_pp
def p_phrase(s,p): return alt(
    p_verb,    ELSE,
    p_noun,    ELSE,
    p_noun_dp, ELSE,
    p_adj
)(s,p)


In [71]:
dict_proper={}
@debug_pp
def p_sentence(s,p):
    first_capital = s[p]=='I' or ch_title in s[p].attrs.changers
    def r_sentence(ph,d):
        rez=StC([I(nodep=ph),I(punct=d)])
        if first_capital: rez.attrs.changers|={ch_sentence}
        return rez
    restore_title=False
    if ch_title in s[p].attrs.changers and s[p] not in dict_proper:
        s[p].attrs.changers-={ch_title}
        restore_title=True
    
    def p_DOT_EXCL(s,p):
        rezs=W('.')(s,p)
        rezs+=W('!')(s,p)
        return rezs
    rezs=seq([p_phrase,p_DOT_EXCL],r_sentence)(s,p)
    
    if restore_title:
        s[p].attrs.changers|={ch_title}
    return rezs

In [72]:
def en2ru(s): # main
    def maxlen_rezs(rezs):
        m=0
        im=set()
        for i in range(len(rezs)):
            if rezs[i][0]>m:
                m=rezs[i][0]
                im={i}
            elif rezs[i][0]==m:
                im.add(i)
        return [rezs[i] for i in im]
                
    s=[ i for i in tokenizer(s)]
    if len(s)==0:
        print('no tokens')
        return ''

    p=0
    rez=[]
    while p<len(s):
        rezs=maxlen_rezs(p_sentence(s,p))
        if len(rezs)==0: break
        p1,r1=next(iter(rezs))
        if len(rezs)>1:
            print('multiple results:')
            print(SAttrs.join(s[p:p1]))
            for void,r in rezs:
                print(str(r))
        p=p1
        rez.append(I(nodep=r1))

    if len(rez)>0:
        if p!=len(s):
            print('NOT PARSED:')
            print(SAttrs.join(s[p:]))
        return str(StC(rez))
    else:
        assert p==0
        rezs=maxlen_rezs(p_phrase(s,p))
        if len(rezs)==0:
            raise ParseError('no results')
        p1,r1=next(iter(rezs))
        if len(rezs)>1:
            print('multiple results:')
            print(SAttrs.join(s[p:p1]))
            for void,r in rezs:
                print(str(r))
        p=p1
        if p!=len(s):
            print('NOT PARSED!')
            print(SAttrs.join(s[p:]))
        return str(r1)


# Тесты

```
местоимения вп
    7)  у меня есть
        у меня нет
        покажи
        дай
    8)  скажи
    9)  (находится) на/в
    10) вопросы-ответы
        считать от до
        ловить
    11) сколько
    13) где
    14) какого цвета
        КОНТЕКСТ
    15) посмотри

watch, двое, трое, пятеро
...
I() - чтобы чтобы модификации свойств записывала в 3й элемент тупла
get_property(tuple)
    если 2 элемента - берет настоящее сойство
    если 3 элемента - берет свойство из 3го элемента
использовать её в init-ах для main и maindep
.pull_deferred():
    с каждым tupl-ом из talk
        если он Struct
            вызываем его.pull_deferred()
        применяем свойства из 3го элемента tupl-а
    подтягиваем .pre из talk[0]
.tistr()
    .pull_deferred
    .__str__

сделать кэширование результатов
    статистика использования паттернов/словарей/правил

атрибуты слов: (теги)
...
нужен LR-парсер, причем недетерминированный - посмотреть, какие есть библиотеки
    возможно стоит придумать уровни для паттернов...
```

In [73]:
def decline(s,pads=['ip','rp','dp','vp','tp','pp']):
    s=[ i for i in tokenizer(s)]
    # добавить дочитывание точки и остаточных пробелов
    rezs=[res for pos,res in p_noun(s,0) if pos==len(s)]
    if len(rezs)!=1:
        raise TextError(rezs)
    tmp=rezs[0]
    
    m=[]
    for p in pads:
        #print(str(tmp))
        prompt= \
            '' if p=='ip' else\
            'нет ' if p=='rp' else\
            'дать ' if p=='dp' else\
            'вижу ' if p=='vp' else\
            'творю ' if p=='tp' else\
            'думаю о ' if p=='pp' else\
            throw(ValueError('bad pad: '+p))
        #rez=deepcopy(tmp)
        tmp.pad=p
        m.append(prompt+str(tmp))#        print(prompt+str(tmp))
    return m


In [74]:
decline('one watch')

не совпадают числа числ. и сущ.: один часы
не совпадают числа числ. и сущ.: один часы
не совпадают числа числ. и сущ.: один часы


['одни часы',
 'нет одних часов',
 'дать одним часам',
 'вижу одни часы',
 'творю одними часами',
 'думаю о одних часах']

In [75]:
en2ru('')

no tokens


''

In [76]:
en2ru('I see jam and one cup')

'я вижу джем и одну чашку'

In [77]:
en2ru('I have a cat.')

'У меня есть кот.'

In [78]:
en2ru('I have no a cat.')

'У меня нет кота.'

In [79]:
en2ru('I have no two guns.')

'У меня нет двух ружей.'

In [80]:
en2ru('have')

'имейте'

In [81]:
en2ru('I have a dog, but I have no cat.')

'У меня есть собака, но у меня нет кота.'

In [82]:
en2ru('I have no lamp.')

'У меня нет лампы.'

In [83]:
en2ru('copy-book')

'тетрадь'

In [84]:
en2ru('give me')

'дай мне'

In [85]:
en2ru('give me cat')

'дай мне кота'

In [86]:
en2ru('to give me cat')

'дать мне кота'

In [87]:
en2ru('I have a kitten, but I have no squirrel.')

'У меня есть котёнок, но у меня нет белки.'

In [88]:
en2ru('Show me Lesson Four!')

'Покажи мне Урок Четыре!'

In [89]:
en2ru('cat has cat')

'у кота есть кот'

In [90]:
en2ru('cat has me')

'у кота есть я'

In [91]:
en2ru('me')

'мне'

In [92]:
en2ru('Good_morning')

'Доброе_утро'

In [93]:
en2ru('good mornings')

'хорошие утра'

In [94]:
en2ru('seven')

'семь'